Stand 10.8.2019, S. Mack
# Messdaten mit einem Arduino Uno aufnehmen und mit Python grafisch animiert darstellen oder in Datei loggen

Für das Erfassen von Sensormesswerten im Rapid Prototyping ist oft ein einfacher Arduino Uno Mikrocontroller völlig ausreichend. Die Messwerte können schon innerhalb der Arduino IDE in Texform und teils auch grafisch dargestellt werden, wenn  sie vom Arduino über die serielle Schnittstelle an den PC übertragen werden.  
Es gibt auch fertige Software, die Messdaten zusätzlich in eine Datei schreibt, also die Funktion eines Datenloggers erfüllt.  

Beides, die animierte grafische Darstellung der Messdaten wie auch das Loggen, kann auch mit Python-Skripten bewerkstelligt werden. Nachfolgend wird jeweils ein Lösungsansatz hierfür inklusive des C-Quellcodes für den Arduino vorgestellt.  

**Wichtige Info:**  
Der Quellcode dieses Jupyter-Notebooks kann nicht innerhalb des Browsers ausgeführt werden. Zum Testen der Beispielprogramme sollte deren Quellcode z.B. in einer Python-Konsole oder unter der Python-IDE Spyder ausgeführt werden.  

Zudem ist ein via USB angeschlossener Arduino mit der entsprechenden Firmware (C-Programm) nötig, dessen Schnittstelleninformation im Python-Code angepasst werden muss. Die vom Arduino im PC belegte Schnittstelle kann man z.B. in der Arduino-IDE nachschauen.

## Darstellung der vom Arduino aufgenommenen Messdaten in einem animierten Python-Plot
Das nachfolgende Codebeispiel basiert auf dem Beispiel "Oscilloscope" aus "Matplotlib User Guide" von J. Hunter et al.  

Zwischen dem PC, der diesen Code ausführt, und dem Arduino wird ein "Handshake" ausgeführt: Erst nachem der PC über die serielle Schnittstelle ein ASCII-Zeichen (hier ``\n`` für einen Zeilenumbruch) gesendet hat, führt die Firmware des Arduino eine AD-Wandlung durch und sendet den Wert des Wandlerergebnisses in Form von ASCII-Zeichen an das Pythonprogramm zurück.  
Dadurch ist anders als bei einem Streamen der Messdaten gewährleistet, dass nur so viele Messdaten gesendet werden, wie die grafische Animation auch entgegen nehmen kann. Die Animation ist hier also der geschwindigkeitsbestimmende Prozess.

Für den nachfolgenden Pythoncode muss auf dem Arduion folgnedes C-Programm hochgeladen werden:  

### C-Programm der Arduino Firmware für den animierten Plot

```c
int value;

void setup() { 
  Serial.begin(115200);          // Seriellle Schnittstelle initialisieren
  Serial.println(0);             // Nötig sonst Fehlfunktion Pythonprogramm, Ursache unklar
}

void loop() {
  if (Serial.available()>0)  {    // Messung vom PC angefordert (Zeichen liegt bei Schnittstelle an)
    int inByte = Serial.read();   // Schnittstelle auslesen  
    value = analogRead(0);        // Signal auf Analog In Pin A0 wandeln    
    Serial.println(value, DEC);   // Wandlerergebnis formatiert ausgeben
  }
}
```

Wenn der Arduino über die serielle Schnittstelle ein Zeichen empfangen hat (egal welches), dann wird das Signal am analogen Eingang A0 AD-gewandelt und anschließend formatiert über die serielle Schnittstelle ausgegeben.  


### Python-Quellcode für den animierten Plot
**Wichtige Info:**  
**Speziell bei der IDE Spyder fun<ktioniert der nachfolgende Code *nicht* in der IPython-Konsole. Daher hier unbedingt in Spyder unter ``Run > Configuration per file...`` die Option ``Execute in an external system terminal`` auswählen.** Nur dann blockiert der Befehl ``plt.show()`` die weitere Ausführung des Programms, so dass das Schließen der seriellen Schnittstelle im ``finally``-Block erst nach dem Schließen des Plotfensters erfolgt.  

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from time import sleep
import serial
from os import _exit


SAMPTIME=1 # Samplingzeit in s
TIMESPAN=10 # Dauer Aufzeichnungsintervalls in s, vor neuer Bildaufbau
YPAD=2 # Y-Randbereich Plot über und unter min/max Daten

adcVal=0
class Scope(object):

    def __init__(self, ax, maxt=10, maxy=100, dt=1): 
        self.ax = ax
        self.dt = dt # Samplingintervall
        self.maxt = maxt # im Plot dargestellte Zeitspanne in s
        self.maxy = maxy # initiale Länge y-Achse, wird später angepasst
        self.tdata = [0]
        self.ydata = [0]
        self.line = Line2D(self.tdata, self.ydata)
        self.ax.add_line(self.line)
        self.ax.set_ylim(0, self.maxy)
        self.ax.set_xlim(0, self.maxt)
        self.ax.set_title('ADC Werte')
        self.ax.set_xlabel('Zeit (s)')
        self.ax.set_ylabel('ADC Rohdaten (LSB)')
        
    def update(self, y):
        lastt = self.tdata[-1]
        # wenn mehr als maxt Sekunden vergangen
        if lastt > self.tdata[0] + self.maxt:
            # jeweils letzes altes Element wird erstes neues Element in Liste
            self.tdata = [self.tdata[-1]]
            self.ydata = [self.ydata[-1]]
            #x-Achse nachskalieren für nächste Zeitspanne
            self.ax.set_xlim(self.tdata[0], self.tdata[0] + self.maxt)
            self.ax.figure.canvas.draw()
        
        t = self.tdata[-1] + self.dt # aktuelle Zeit
        self.tdata.append(t) # Liste Zeiten um neue Messzeit erweitern
        self.ydata.append(y) # Liste Werte um neuen Messwert erweitern
        self.ax.set_ylim(min(self.ydata)-YPAD, max(self.ydata)+YPAD)
        self.line.set_data(self.tdata, self.ydata)
        return self.line,

def getAdcVal(): # neuen Datenpunkt von serieller Schnittstelle lesen
    port.flushInput() # seriellen Puffer leeren
    port.write(b'\n') # ADC-Messung Arduino anfordern
    result = int(port.readline()) # von Schnittstelle bis \n lesen (Byte Array)
    return(result)

def data_gen(): # muss Iterator sein, daher die While-Schleife
    while True:
        adcVal = getAdcVal()
        yield adcVal    
      
try:
    port = serial.Serial('COM22', 115200) # Hier richtige Schnittstelle/Baudrate angeben
    fig, ax = plt.subplots() # Plot und Achse erstellen
    scope = Scope(ax=ax,maxt=TIMESPAN,dt=SAMPTIME)
    # Animation instanzieren, intervall in ms, blit=False zum Nachskalieren x-Achse nötig 
    ani = animation.FuncAnimation(fig, scope.update, data_gen, interval=(SAMPTIME*1000), blit=False)
    plt.show() # Plot starten (Programm bleibt ab jetzt in dieser Schleife bis Plotfenster geschlossen)     
except KeyboardInterrupt:
    print()
    print('Strg + C erkannt...')
    sleep(2)      
finally: # wird ausgeführt nach Schließen des Plotfensters
    print('...COM-Verbindung wird beendet.')
    port.close() # COM-Verbindung beenden
    sleep(1)
    _exit(1) # Konsolenfenster schließen

## Loggen der vom Arduino aufgenommenen und gestreamten Messdaten in eine Datei
Das Ziel des nachfolgenden Arduino C-Code ist es, Messdaten *mit maximaler Rate* zu erfassen und diese *so schnell wie möglich* über die serielle Schnittstelle an einen PC zu übertragen.  
Um die maximal mögliche Geschwindigkeit zu erreichen, werden im Gegensatz zum animierten Plot hier die Daten gestreamt (also kein Handshake) und binär (d.h. nicht als ASCII-Zeichen) übertragen.  

### C-Programm der Arduino-Firmware für das Daten-Loggen
Anstelle von Sensormessdaten wird in diesem Beispiel-C-Code der aktuelle Stand des Mikrosekundenzählers fortlaufend übertragen. Dadurch kann später in der Log-Datei die erreichte Datenrate einfach ermittelt werden, denn jede Zeile entspricht einem neuen Schleifendurchlauf.

```c
unsigned char valByte[2]={0x00,0x00}; // Zwei Bytes, welche seriell binär übertragen werden
unsigned long maskLowByte=0x000000FF; // Bitmaske, um LSB 0-7 aus Zählerwert zu extrahieren

unsigned long timeNow=0;

void setup() {   
  Serial.begin(2000000);
  delay(500);
  Serial.println(9999999); // Nötig sonst Fehlfunktion Pythonprogramm, Ursache unklar
  delay(500);
}

void loop() {
  timeNow = micros(); // Zählerwert auslesen
  valByte[0] = timeNow & maskLowByte; // niederwertiges Byte (Position 0)
  valByte[1] = (timeNow >> 8) & maskLowByte; // höherwertiges Byte (Position 1)
  Serial.write(valByte,2); // Zwei Elemente des Arrays binär über die Schnittstelle senden
}
```

Der Arduino gibt fortlaufend den Wert des Zählers ``micros()`` über die serielle Schnittstelle aus, ohne vorher auf ein Handshake-Signal vom PC zu warten. Die beiden niederwertigsten Bytes des 32-Bit-Zählerwerts ``micros()`` werden als zwei getrennte Bytes über die serielle Schnittstelle gesendet.  

### Python-Quellcode für das Daten-Loggen
Der folgende Python-Code nimmt die vom Arduino gestreamten Zählerwerte entgegen:  
Zuerst wird zur Kontrolle (und als work around eines noch nicht gelösten Bugs) der erste Wert ``9999999`` vom Arduino gelesen. Danach werden jeweils zwei Bytes hintereinander von der seriellen Schnittstelle gelesen. Diese werden anschließend in einen Integerwert konvertiert, in einer Liste zwischengespeichert und am Ende in eine Log-Datei (hier ``testDatei.txt``) geschrieben.  
Es wird nicht kontrolliert, ob die Reihenfolge der beiden Bytes (zuerst Low und dann High Byte) beim Lesen korrekt ist.  
Dieser Quellcode kann anders als beim animierten Plot auch in einer IPython-Konsole beispielsweise von Spyder ausgeführt werden.

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
import serial
from time import sleep

print('Info: Abbruch mit strg + c')
data = []

try:
    port = serial.Serial('COM22', 2000000)
    sleep(0.5)
    port.flushInput() # seriellen Puffer leeren
    sleep(1)
    print('Start Datenaufzeichung in Datei testDatei.txt in 0,5 Sekunden...')
    # prüfen ob Arduino bereit, Zahl 9999999 sollte empfangen werden
    print('Arduino Kontrollzahl gelesen (9999999): ', int(port.readline())) 
    sleep(0.5) # dem seriellen Puffer etwas Vorsprung geben
    for i in range(100):
       numBin = port.read(2) # Byte Array mit zwei Elementen (High und Low Byte)
       value = int(numBin[0]+256*numBin[1])
       data.append(value)
except KeyboardInterrupt:
    print('Programm mit strg + c abgebrochen bzw. Laufzeitfehler')  
finally:
    f = open("testDatei.txt", "w")
    for item in data: 
        f.write(str(int(item))+'\n')
    f.close()
    print('Datei geschlossen')
    port.close() # serielle Schnittstelle schliessen
    print('Schnittstelle geschlossen')

Auszug aus der Log-Datei ``testDatei.txt``:
```
17308
17328
17348
17368
17388
17412
17432
17452
17472
17492
17512
17532
17552
17568
17588
17608
...
```

Wie in der Log-Datei oben zu sehen ist, werden die Zählerstände alle 20 µs, also mit einer Rate von 50 kHz vom Arduino auf die serielle Schnittstelle ausgegeben. **Die Schnittstelle selbst mit der eingestellten Baudrate von 2.000.000 Baud ist bei dieser Übertragungsrate noch nicht am Limit, wie folgende Abschätzung zeigt:**  
2.000.000 Baud ergibt eine Bruttoübertragungsrate von 2.000.000 / 16 für die 16 Bit Zählerwerte. Es exisitieren jedoch noch 
zwei Framing Bits zusätzlich zu den je acht Datenbits (8-N-1). Dies führt zu einer 20 % geringeren Nettoübertragungsrate von 0.8 \* 2.000.000 Baud / 16 = 100 kHz.  
Somit ist die Rechenzeit des Arduinos für die gemessene Rate von 50 kHz verantwortlich. Dies ist auch zwingend notwendig, denn sonst würde der Ausgabepuffer der seriellen Schnittstelle am Arduino überlaufen.  
Das Pythonprogramm auf dem PC kann offentlichtlich die an der seriellen Schnittstelle anliegenden Daten schnell genug in die Liste ``data[]`` abspeichern. Sonst würde am PC der Eingangspuffer überlaufen.